<a href="https://colab.research.google.com/github/takitakir/JOBS/blob/main/Geez%20done.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import csv

In [2]:
site_map = "https://geezjobs.com/jobs-in-ethiopia"

In [8]:
base_url = "https://geezjobs.com"

In [7]:
jobs_list_url = "https://geezjobs.com/jobs-in-ethiopia"

In [3]:
response = requests.get(site_map)

In [4]:
html = response.text

In [5]:
soup = BeautifulSoup(html, "html.parser")

In [10]:
def scrape_job_listings():
    response = requests.get(jobs_list_url)
    soup = BeautifulSoup(response.text, "html.parser")

    jobs = []

    # Find all job list items
    job_items = soup.find_all('li')

    for item in job_items:
        try:
            # Skip if not a job item (doesn't contain the expected structure)
            if not item.find('a', class_='color-green'):
                continue

            # Extract job title and URL
            title_tag = item.find('a', class_='color-green')
            job_title = title_tag.text.strip()
            relative_url = title_tag['href']
            job_url = base_url + relative_url if relative_url.startswith('/') else relative_url

            # Extract company name
            company_tag = item.find('a', href=lambda x: x and '/company/' in x)
            company = company_tag.text.strip() if company_tag else "N/A"

            # Extract location
            location_tag = item.find('span', string=lambda x: x and '<i class="fa fa-map-marker"></i>' in str(x))
            location = location_tag.text.strip() if location_tag else "N/A"

            # Extract closing date
            closing_tag = item.find('code')
            closing_text = closing_tag.text.strip() if closing_tag else "N/A"
            closing_date = parse_closing_date(closing_text)

            jobs.append({
                'title': job_title,
                'company': company,
                'location': location,
                'closing_date': closing_date,
                'closing_days_text': closing_text,
                'url': job_url
            })

        except Exception as e:
            print(f"Error processing a job listing: {e}")
            continue

    return jobs

In [11]:
def parse_closing_date(days_text):
    """Convert 'X days left' to actual deadline date"""
    try:
        if 'day' in days_text.lower():
            days = int(days_text.split()[0])
            return (datetime.now() + timedelta(days=days)).strftime('%Y-%m-%d')
        return days_text
    except:
        return days_text

In [12]:
def save_to_csv(jobs, filename='geezjobs_jobs.csv'):
    with open(filename, 'w', newline='', encoding='utf-8') as f:
        fieldnames = ['title', 'company', 'location', 'closing_date', 'closing_days_text', 'url']
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(jobs)

In [13]:
if __name__ == '__main__':
    print("Scraping job listings from GeezJobs...")
    jobs = scrape_job_listings()

    save_to_csv(jobs)
    print(f"Successfully scraped {len(jobs)} jobs. Data saved to geezjobs_jobs.csv")

Scraping job listings from GeezJobs...
Successfully scraped 30 jobs. Data saved to geezjobs_jobs.csv


In [14]:
jobs

[{'title': 'Executive Sous Chef',
  'company': 'Ethiopian Airlines',
  'location': 'N/A',
  'closing_date': '2025-05-22',
  'closing_days_text': '7 days left',
  'url': 'https://geezjobs.com/job-detail/executive-sous-chef-ethiopian-airlines-2'},
 {'title': 'Junior Accountant',
  'company': 'Joytech PLC',
  'location': 'N/A',
  'closing_date': '2025-05-29',
  'closing_days_text': '14 days left',
  'url': 'https://geezjobs.com/job-detail/junior-accountant-joytech-plc'},
 {'title': 'Jr. Driver',
  'company': 'Ethiopian Airlines',
  'location': 'N/A',
  'closing_date': '2025-05-22',
  'closing_days_text': '7 days left',
  'url': 'https://geezjobs.com/job-detail/jr-driver-ethiopian-airlines'},
 {'title': 'IT Field Engineer',
  'company': 'East Africa Bottling Share Company - Coca Cola',
  'location': 'N/A',
  'closing_date': '2025-05-26',
  'closing_days_text': '11 days left',
  'url': 'https://geezjobs.com/job-detail/it-field-engineer-east-africa-bottling-share-company-coca-cola'},
 {'titl